In [2]:
%load_ext autoreload
%autoreload 2
from notebook import *
# if get something about NUMEXPR_MAX_THREADS being set incorrectly, don't worry.  It's not a problem.

# Myth of programming

Consider the following code snippets, which side is faster?

In [3]:
compare([do_render_code("swap.c", show="regswap"),do_render_code("swap.c", show="xorswap")])

In [5]:
render_code("swap.c")

// swap.c:1-89 (89 lines)
#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>

void (*swap)(int* a, int* b);
void regswap(int* a, int* b)
{
    int temp = *a;
    *a = *b;
    *b = temp;
}

void xorswap(int* a, int* b)
{
    *a ^= *b;
    *b ^= *a;
    *a ^= *b;
}

/* This function takes last element as pivot, places
   the pivot element at its correct position in sorted
    array, and places all smaller (smaller than pivot)
   to left of pivot and all greater elements to right
   of pivot */
int partition (int arr[], int low, int high)
{
    int pivot = arr[high];    // pivot
    int i = (low - 1);  // Index of smaller element
 
    for (int j = low; j <= high- 1; j++)
    {
        // If current element is smaller than or
        // equal to pivot
        if (arr[j] <= pivot)
        {
            i++;    // increment index of smaller element
            if(i!=j)
            swap(&arr[i], &arr[j]);
        }
    }
            if(i+1 != high)
    swap(&arr[i + 1], &arr[high]);
    return (i + 1);
}
 
/* The main function that implements QuickSort
 arr[] --> Array to be sorted,
  low  --> Starting index,
  high  --> Ending index */
void quickSort(int arr[], int low, int high)
{
    if (low < high)
    {
        /* pi is partitioning index, arr[p] is now
           at right place */
        int pi = partition(arr, low, high);
 
        // Separately sort elements before
        // partition and after partition
        quickSort(arr, low, pi - 1);
        quickSort(arr, pi + 1, high);
    }
}
 

int main(int argc, char **argv)
{
    unsigned array_size = 131072;
    int *data;
    struct timeval time_start, time_end;
    // command: swap array_size method
    if(argc > 1)
        array_size = atoi(argv[1]);
    data = (int *)malloc(sizeof(int)*array_size);
    if(argc > 2 && atoi(argv[2])==1)
        swap = xorswap;
    else
        swap = regswap;
    for (unsigned i = 0; i < array_size; ++i)
        data[i] = rand();
   gettimeofday(&time_start, NULL);
   quickSort(data, 0, array_size-1);
/*   for (unsigned i = 0; i < array_size; ++i)
        swap(&data[i],&data[array_size-1-i]);*/
   gettimeofday(&time_end, NULL);
   fprintf(stderr, "data[rand()] = %d\n", data[rand()%array_size]);
   fprintf(stderr, "sorted %lf seconds\n",((time_end.tv_sec * 1000000 + time_end.tv_usec) - (time_start.tv_sec * 1000000 + time_start.tv_usec))/1000000.0);
   return 0;
}

In [6]:
! make clean; make swap
! echo "Version A"
! ./swap 1048576 0; ./swap 1048576 0;  ./swap 1048576 0;  ./swap 1048576 0;  ./swap 1048576 0
! echo "Version B"
#./arithmetics_2_B; ./arithmetics_2_B; ./arithmetics_2_B; ./arithmetics_2_B; ./arithmetics_2_B
! ./swap 1048576 1;./swap 1048576 1;./swap 1048576 1;./swap 1048576 1;./swap 1048576 1

rm -f arithmetics arithmetics_A arithmetics_B linked_list flip_array swap linked_list.s
cc -O3 -DHAVE_LINUX_PERF_EVENT_H swap.c -o swap
Version A
data[rand()] = 963389671
sorted 0.062791 seconds
data[rand()] = 963389671
sorted 0.056405 seconds
data[rand()] = 963389671
sorted 0.057624 seconds
data[rand()] = 963389671
sorted 0.056913 seconds
data[rand()] = 963389671
sorted 0.056685 seconds
Version B
data[rand()] = 963389671
sorted 0.059007 seconds
data[rand()] = 963389671
sorted 0.057761 seconds
data[rand()] = 963389671
sorted 0.058260 seconds
data[rand()] = 963389671
sorted 0.058288 seconds
data[rand()] = 963389671
sorted 0.058153 seconds


In [ ]:
! gcc -S -O3 swap.c

In [ ]:
render_code("swap.s")

## Linked List v.s. Array

In [ ]:
compare([do_render_code("linked_list.c", show="sum_of_array"),do_render_code("linked_list.c", show="depth_of_list")])

In [ ]:
! make clean; make
! echo "size,list,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses" > stats.csv
! echo -n "4096,1," >> stats.csv
! ./linked_list 4096 1
! echo -n "4096,0," >> stats.csv
! ./linked_list 4096 0

In [ ]:
display_df_mono(render_csv("stats.csv"))

In [ ]:
render_code("linked_list.s", show=["sum_of_array:",".LFE40:"])

The main loop between L10 and L9 has two 